# Note Book To Fix Country Boarder Problem

This notebook is to clip polygons that overlap country boundaries

By Cascade Tuholske and Ryan Avery 2019-04-29

### Dependencies

In [1]:
from rasterstats import zonal_stats
import rasterio
import geopandas as gpd
import operator
import numpy as np
import pandas as pd
import matplotlib 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from functools import reduce
import squarify
from scipy import stats as ss

### Load Data

In [2]:
# Paths
data_raw = '../../data/raw/'
data_temp = '../../temp_data/'
data_interim = '../../interim/'
erl_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/ERL_data/'
downloads = '/Users/cascade/Downloads/'
erl_v2_data = '/Users/cascade/Github/Pop-ERL/temp_data/ERL19/ERLv2/'

In [3]:
GHS2000 = gpd.read_file(erl_v2_data+'GHS2000_20190429_all.shp', driver = 'ESRI Shapefile')
GHS2015 = gpd.read_file(erl_v2_data+'GHS2015_20190429_all.shp', driver = 'ESRI Shapefile')
WP2015 = gpd.read_file(erl_v2_data+'WP2015_20190429_all.shp', driver = 'ESRI Shapefile')
LS2015 = gpd.read_file(erl_v2_data+'WPE2016_20190429_all.shp', driver = 'ESRI Shapefile')
WPE2016 = gpd.read_file(erl_v2_data+'LS2015_20190429_all.shp', driver = 'ESRI Shapefile')

In [8]:
def find_dup(df, col):
    df_out = gpd.GeoDataFrame()
    duplicated = df[col].duplicated()
    
    df_out = df[duplicated]
    return df_out

In [106]:
num = [1,1,2,3,3,1]
state = ['mt', 'ca', 'mt', 'pa', 'pa', 'mt']
cool = ['yes', 'yes', 'yes', 'no', 'yes', 'yes']

test = pd.DataFrame()
test['num'] = num
test['state'] = state
test['cool'] = cool


test

,num,state,cool
0,1,mt,yes
1,1,ca,yes
2,2,mt,yes
3,3,pa,no
4,3,pa,yes
5,1,mt,yes


In [110]:
out = test.groupby(['state', 'num']).groups

In [111]:
for key, value in out:
    print(key, value)

ca 1
mt 1
mt 2
pa 3


In [107]:
g = test.groupby('num')['state'].apply(np.unique)
g

num
1    [ca, mt]
2        [mt]
3        [pa]
Name: state, dtype: object

In [101]:
test.groupby(['state','num']).size().reset_index().rename(columns={0:'count'})

(5854, 10)

In [102]:
test = find_dup(GHS2000, 'FID')

In [105]:
test.groupby(['FID','country'])

In [12]:
out = test.groupby('FID')['country'].apply(np.unique)

In [113]:
test = find_dup(GHS2000, 'FID')

out = test.groupby(['country', 'FID']).groups
for key, value in out:
    print(key, value)

Algeria 54
Algeria 138
Algeria 172
Algeria 197
Algeria 232
Algeria 240
Algeria 246
Algeria 257
Algeria 327
Algeria 329
Algeria 368
Algeria 435
Algeria 437
Algeria 467
Algeria 518
Algeria 556
Algeria 614
Algeria 619
Algeria 624
Algeria 712
Algeria 754
Algeria 766
Algeria 772
Algeria 836
Algeria 843
Algeria 1043
Algeria 1065
Algeria 1223
Algeria 1276
Algeria 1377
Algeria 1415
Algeria 1478
Algeria 1712
Algeria 2210
Algeria 3046
Algeria 3134
Algeria 3566
Algeria 3798
Angola 36450
Angola 36509
Angola 37544
Angola 38670
Angola 38879
Angola 39782
Angola 39961
Benin 15600
Benin 21647
Benin 22327
Benin 23165
Benin 23347
Botswana 43675
Burkina Faso 11146
Burkina Faso 12993
Burkina Faso 14200
Burundi 33762
Cameroon 15285
Cameroon 24910
Cameroon 26543
Cameroon 26570
Cameroon 26677
Cape Verde 5507
Central African Republic 24661
Central African Republic 26404
Chad 11838
Comoros 38746
Congo-Brazzaville 28047
Congo-Brazzaville 34633
Côte d'Ivoire 25425
Djibouti 13083
Egypt 4235
Egypt 4256
Egypt 4341
E